In [37]:
import numpy as np
import pandas as pd
from itertools import product

import itertools
import numpy as np
import pandas as pd

In [36]:
#Code für T=2

In [2]:
import pandas as pd
import numpy as np

# Modellparameter
r = 0.1
discount_factor = (1 + r)**2  # 1.21
S0 = 1
K = 1

# Trinomialpfade für zwei Perioden
states_t2 = [
    ('u', 'u'), ('u', 'm'), ('u', 'd'),
    ('m', 'u'), ('m', 'm'), ('m', 'd'),
    ('d', 'u'), ('d', 'm'), ('d', 'd')
]

# Bewegungsfaktoren
factors = {'u': 2.0, 'm': 1.0, 'd': 0.1}
probs = {'u': 0.2, 'm': 0.5, 'd': 0.3}

# Berechne Pfadinformationen
data = []

for path in states_t2:
    p1, p2 = path
    S2 = S0 * factors[p1] * factors[p2]
    discounted_S2 = S2 / discount_factor
    payoff = max(S2 - K, 0)
    discounted_payoff = payoff / discount_factor
    prob = probs[p1] * probs[p2]
    data.append({
        "Path": f"{p1}{p2}",
        "S2": round(S2, 3),
        "tilde_S2": round(discounted_S2, 3),
        "Payoff": round(payoff, 3),
        "tilde_X": round(discounted_payoff, 3),
        "P(path)": round(prob, 3)
    })

df_t2 = pd.DataFrame(data)

df_t2


,Path,S2,tilde_S2,Payoff,tilde_X,P(path)
0,uu,4.00,3.306,3.0,2.479,0.04
1,um,2.00,1.653,1.0,0.826,0.10
2,ud,0.20,0.165,0.0,0.000,0.06
3,mu,2.00,1.653,1.0,0.826,0.10
4,mm,1.00,0.826,0.0,0.000,0.25
5,md,0.10,0.083,0.0,0.000,0.15
6,du,0.20,0.165,0.0,0.000,0.06
7,dm,0.10,0.083,0.0,0.000,0.15
8,dd,0.01,0.008,0.0,0.000,0.09


In [30]:
#Berechnung von E[x|F_1] und der oprimalen position \xi_2 an den Knoten t=1

In [3]:
# Gruppiere Pfade nach den ersten Knoten (t = 1), um Erwartungswerte unter F1 zu berechnen
from collections import defaultdict

# Erstelle eine Struktur, die Gruppen nach erstem Zeitschritt sammelt
grouped_by_t1 = defaultdict(list)

for row in data:
    first, second = row["Path"][0], row["Path"][1]
    grouped_by_t1[first].append(row)

# Berechne für jede Gruppe: E[tilde{X}], E[Delta tilde{S}_2], Cov, Var
hedging_t1 = []

for state, paths in grouped_by_t1.items():
    E_X = sum(p["tilde_X"] * p["P(path)"] for p in paths)
    E_dS = sum((p["tilde_S2"] - factors[state] / (1 + r)) * p["P(path)"] for p in paths)
    E_X_dS = sum(p["tilde_X"] * (p["tilde_S2"] - factors[state] / (1 + r)) * p["P(path)"] for p in paths)
    Cov_X_dS = E_X_dS - E_X * E_dS
    Var_dS = sum(((p["tilde_S2"] - factors[state] / (1 + r))**2) * p["P(path)"] for p in paths) - E_dS**2
    xi_2 = Cov_X_dS / Var_dS if Var_dS != 0 else 0
    hedging_t1.append({
        "State at t=1": state,
        "E[tilde{X}]": round(E_X, 3),
        "E[Delta tilde{S}_2]": round(E_dS, 3),
        "Var[Delta tilde{S}_2]": round(Var_dS, 3),
        "Cov(tilde{X}, Delta tilde{S}_2)": round(Cov_X_dS, 3),
        "xi_2": round(xi_2, 3)
    })

df_hedge_t1 = pd.DataFrame(hedging_t1)

df_hedge_t1

,State at t=1,E[tilde{X}],E[Delta tilde{S}_2],Var[Delta tilde{S}_2],"Cov(tilde{X}, Delta tilde{S}_2)",xi_2
0,u,0.182,-0.056,0.252,0.144,0.572
1,m,0.083,-0.070,0.154,0.067,0.435
2,d,0.000,-0.004,0.001,0.000,0.000


In [4]:
# Erwartete Werte für t=1 für jeden Zustand: u, m, d
# Diskontierter Preis des Underlyings nach einem Schritt
S1_discounted = {k: v / (1 + r) for k, v in factors.items()}  # tilde{S}_1

# Nutze Erwartungswerte aus vorheriger Berechnung (E[tilde{X}])
E_tildeX_t1 = {row["State at t=1"]: row["E[tilde{X}]"] for row in hedging_t1}

# Wahrscheinlichkeiten für Zustand bei t=1
P_t1 = {'u': 0.2, 'm': 0.5, 'd': 0.3}

# Berechne Erwartungswerte und Kovarianzen für xi_1
E_X1 = sum(P_t1[state] * E_tildeX_t1[state] for state in P_t1)
E_dS1 = sum(P_t1[state] * (S1_discounted[state] - 1) for state in P_t1)
E_X1_dS1 = sum(P_t1[state] * E_tildeX_t1[state] * (S1_discounted[state] - 1) for state in P_t1)
Cov_X1_dS1 = E_X1_dS1 - E_X1 * E_dS1
Var_dS1 = sum(P_t1[state] * (S1_discounted[state] - 1)**2 for state in P_t1) - E_dS1**2

xi_1 = Cov_X1_dS1 / Var_dS1 if Var_dS1 != 0 else 0
V0_discounted = E_X1
V0 = (1 + r) * V0_discounted
xi0_0 = V0_discounted - xi_1 * 1  # tilde{S}_0 = 1

hedge_t0_result = {
    "E[tilde{X}_1]": round(E_X1, 3),
    "E[Delta tilde{S}_1]": round(E_dS1, 3),
    "Var[Delta tilde{S}_1]": round(Var_dS1, 3),
    "Cov(tilde{X}, Delta tilde{S}_1)": round(Cov_X1_dS1, 3),
    "xi_1": round(xi_1, 3),
    "xi_0^0": round(xi0_0, 3),
    "tilde{V}_0": round(V0_discounted, 3),
    "V_0": round(V0, 3)
}

pd.DataFrame([hedge_t0_result])


,E[tilde{X}_1],E[Delta tilde{S}_1],Var[Delta tilde{S}_1],"Cov(tilde{X}, Delta tilde{S}_1)",xi_1,xi_0^0,tilde{V}_0,V_0
0,0.078,-0.155,0.362,0.038,0.105,-0.027,0.078,0.086


In [5]:
# Berechne Residualkomponenten L2 = tilde{X} - xi_2 * Delta tilde{S}_2
# Dafür benötigen wir die Werte für Delta tilde{S}_2 = tilde{S}_2 - tilde{S}_1 (abhängig vom Zustand bei t=1)

# Zuerst: Berechne tilde{S}_1 für jeden Pfad, abhängig vom ersten Zustand
tilde_S1 = {'u': 2.0 / (1 + r), 'm': 1.0 / (1 + r), 'd': 0.1 / (1 + r)}  # = S1 / 1.1

residuals = []
for row in data:
    path = row["Path"]
    t1_state = path[0]
    tilde_S2 = row["tilde_S2"]
    delta_tilde_S2 = tilde_S2 - tilde_S1[t1_state]
    xi_2 = {'u': 0.572, 'm': 0.435, 'd': 0.000}[t1_state]
    tilde_X = row["tilde_X"]
    L2 = tilde_X - xi_2 * delta_tilde_S2
    residuals.append({
        "Path": path,
        "tilde_X": tilde_X,
        "Delta tilde{S}_2": round(delta_tilde_S2, 3),
        "xi_2": xi_2,
        "L2": round(L2, 3),
        "P(path)": row["P(path)"]
    })

df_residuals = pd.DataFrame(residuals)
df_residuals


,Path,tilde_X,Delta tilde{S}_2,xi_2,L2,P(path)
0,uu,2.479,1.488,0.572,1.628,0.04
1,um,0.826,-0.165,0.572,0.920,0.10
2,ud,0.000,-1.653,0.572,0.946,0.06
3,mu,0.826,0.744,0.435,0.502,0.10
4,mm,0.000,-0.083,0.435,0.036,0.25
5,md,0.000,-0.826,0.435,0.359,0.15
6,du,0.000,0.074,0.000,0.000,0.06
7,dm,0.000,-0.008,0.000,0.000,0.15
8,dd,0.000,-0.083,0.000,0.000,0.09


In [6]:
# Erwartungswert von L2 (L_0)
L2_total = sum(row["L2"] * row["P(path)"] for row in residuals)

# Globaler quadratischer Fehler: E[(L2 - tilde{V}_0)^2]
tilde_V0 = 0.078
global_quadr_error = sum(row["P(path)"] * (row["L2"] - tilde_V0)**2 for row in residuals)

pd.DataFrame([{
    "L_0 = E[L2]": round(L2_total, 3),
    "tilde{V}_0": round(tilde_V0, 3),
    "Global Quadratic Hedging Error": round(global_quadr_error, 4)
}])


,L_0 = E[L2],tilde{V}_0,Global Quadratic Hedging Error
0,0.327,0.078,0.2443
